# Earth's Energy Balance

Ruth Moorman adapted from on CSS Topic 1 (for Joao Teixeira ESE144 Lecture and Graeme Stephens ESE144 Lecture)


<div class="alert alert-block alert-info">
<b>DATA ACCESS:</b>
<div> 
<div>        
    
We'll be working with the Clouds and the Earth's Radiant Energy System project EBAF-TOA Level 3b data sourced from <a href="https://ceres.larc.nasa.gov/data/">here</a>. Information on the instruments used to construct this data product can be found <a href="https://ceres.larc.nasa.gov/instruments/">here</a> and also note that we are working with the "Energy Balanced and Filled" (EBAF) product, methodology details may be found <a href="https://journals.ametsoc.org/view/journals/clim/31/2/jcli-d-17-0208.1.xml">here</a>.
<div><div>

    




In [ ]:
import numpy as np
import xarray as xr
# need to import some plotting tools...
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
import cartopy.feature
import matplotlib.path as mpath

In [ ]:
ds = xr.open_dataset('~/shared/notebooks/CaltechESE1442024/data/CERES_EBAF-TOA_Ed4.2_Subset_200003-202401.nc')
ds

Net radiative flux at TOA is calculated from:

$$\Delta F=F^{\downarrow}_{SW} - F^{\uparrow}_{SW} - F^{\uparrow}_{LW}$$

Where $F^{\downarrow}_{SW}-F^{\uparrow}_{SW}$ is the net incoming shortwave radiation and $F^{\uparrow}_{LW}$ is the outgoing longwave radiation at TOA respectively. 

In [ ]:
rsdt = ds.solar_mon
rlut = ds.toa_lw_all_mon
rsut = ds.toa_sw_all_mon
rlut_clear = ds.toa_lw_clr_c_mon
rsut_clear = ds.toa_sw_clr_c_mon

In [ ]:
absorbed_shortwave_time_mean = (rsdt - rsut).mean({'lon', 'time'})
outgoing_longwave_time_mean  = rlut.mean({'lon', 'time'})
net_energy_input = absorbed_shortwave_time_mean-outgoing_longwave_time_mean

In [ ]:
fig,(ax,ax1) = plt.subplots(1,2, figsize=(12,10))
ax.plot(absorbed_shortwave_time_mean,absorbed_shortwave_time_mean.lat, 'r-',label = 'Absorbed Shortwave')
ax.plot(outgoing_longwave_time_mean,outgoing_longwave_time_mean.lat, 'b-',label = 'Outgoing Longwave')
ax.fill_betweenx(absorbed_shortwave_time_mean.lat, absorbed_shortwave_time_mean,outgoing_longwave_time_mean, color='g', alpha=0.2)
ax.legend(loc=0,fontsize=12)
ax.set_xlabel('Net TOA Flux (W/m2)',fontsize=12);ax.set_ylabel('Latitude',fontsize=12)
ax.set_title('CERES Net Radiative Flux at TOA, \ntime mean (2000/03-2024/01) and zonal mean', fontsize=12)
ax.set_ylim([-90,90])

ax1.plot(net_energy_input,absorbed_shortwave_time_mean.lat, color='g',label = 'Implied Transport')
ax1.axvline(0, color='k',linestyle=':')
ax1.set_title('Imbalance \n (Absorbed Shortwave - Outgoing Longwave)', fontsize=12)
ax1.set_ylim([-90,90])
plt.show()


<div class="alert alert-block alert-info">
<b>TASK:</b>
<div> 
<div>        
    
Plot the same using clear sky values (rlut_clear and rsut_clear).<div><div>

    





<div class="alert alert-block alert-info">
<b>DISCUSS:</b>
<div> 
<div>        
    
Discuss the discrepencies between meridional patterns derived from the clear sky and full dataset.<div><div>

    




### Meridional Energy Transport

From this information, we want to compute the combined meridional energy transport of the Earth's atmosphere and oceans.

As outlines by Graeme earlier in the course, the net energy budget of a column (of here, a latitudinal band) of the Earth can be expressed as 
$$\frac{\partial E}{\partial t} = F_\text{net, TOA} - \mathbf{div}{T_A} - \mathbf{div}{T_O}$$
where $\frac{\partial E}{\partial t} $ is the net accummulation of energy in the column, $F_\text{net, TOA}$ is the net energy input (obtained above) and $-\mathbf{div}{T_A} - \mathbf{div}{T_O}$ is the divergence of energy from the column by the atmosphere and oceans.

For this problem, we assume the time mean (here we're averaging over 24 years) storage of energy in the column is zero. Then we can rewrite the abonve into an expression for the divergence of energy by the oceans and atmosphere (note $\phi$ is latitude),

$$ \langle F_\text{net,TOA}\rangle = \langle \mathbf{div}{T_A} \rangle + \langle \mathbf{div}{T_O}\rangle = \frac{1}{a}\frac{\partial T_{A+O}}{\partial\phi}$$
$$ ...$$
$$T_{A+O}(\phi) = 2\pi R_E^2\int_{\phi=\pi/2}^{\phi}\langle F_\text{net,TOA}(\phi')\rangle \, \cos{(\phi')}\, d\phi'.$$

We can discretize this integral as,

$$ T({\phi_0}) = \text{Area}(\phi_0,\phi_1) \times F_{net}(\phi_0,\phi_1)$$
$$ T({\phi_1}) = \text{Area}(\phi_0,\phi_1) \times F_{net}(\phi_0,\phi_1)+ \text{Area}(\phi_1,\phi_2) \times F_{net}(\phi_1,\phi_2)$$
$$ ...$$

AKA **a cumulative sum through latitude of the net energy input times the band area.**



<div class="alert alert-block alert-info">
<b>TASK:</b>
<div> 
<div>        
    
    Compute and plot the transport term above.

<div>
    


<div class="alert alert-block alert-info">
<div> 
<div>        
    
    To do this, we'll first need to approximate the area in each latitude band. This can be expressed,
    $$A = R_E^2 (\sin(\phi_2) - \sin(\phi_1))$$
where latitude values are in radians.

In [ ]:
# where's not given latitude grid cell bounds here explicitly but the grid is very even so this is simple
print(net_energy_input.lat.values)

In [ ]:
## creat a band area array (units m)
R = 6378 * 1e3
lat_lower = ## your code
lat_upper = ## your code
A_band = ## your code


<div class="alert alert-block alert-info">
<div> 
<div>        
    
Then use this area array to approximate the integral.    
<div>
    

In [ ]:
meridional_energy_transport = ## your code

In [ ]:
fig,(ax,ax1) = plt.subplots(1,2, figsize=(12,10))
ax.plot(absorbed_shortwave_time_mean,absorbed_shortwave_time_mean.lat, 'r-',label = 'Absorbed Shortwave')
ax.plot(outgoing_longwave_time_mean,outgoing_longwave_time_mean.lat, 'b-',label = 'Outgoing Longwave')
ax.fill_betweenx(absorbed_shortwave_time_mean.lat, absorbed_shortwave_time_mean,outgoing_longwave_time_mean, color='g', alpha=0.2)
ax.legend(loc=0,fontsize=12)
ax.set_xlabel('Net TOA Flux (W/m2)',fontsize=12);ax.set_ylabel('Latitude',fontsize=12)
ax.set_title('CERES Net Radiative Flux at TOA, \ntime mean (2000/03-2024/01) and zonal mean', fontsize=12)
ax.set_ylim([-90,90])

# ax1.plot(meridional_energy_transport,meridional_energy_transport.lat, color='g',label = 'Implied Transport')
ax1.axvline(0, color='k',linestyle=':')
ax1.set_title('Meridional Energy Transport', fontsize=12)
ax1.set_xlabel('W',fontsize=12)
ax1.set_ylim([-90,90])
plt.show()


<div class="alert alert-block alert-info">
<b>DISCUSS:</b>
<div> 
<div>        
   
Discuss the features of this figure. Is it hemispehrically symmetric? Why/Why not? Where does the transport go to zero? Why? 
<div>
    


<div class="alert alert-block alert-info">
<b>TASK:</b>
<div> 
<div>        
    
Repeat this analysis using clear sky values (rlut_clear and rsut_clear).<div><div>

    





<div class="alert alert-block alert-info">
<b>DISCUSS:</b>
<div> 
<div>        
    
Why doesn't this work? What assumptions have been broken?<div><div>

    


